In [13]:
# !pip install --upgrade tweeterpy
# !pip install gspread_dataframe 

In [1]:
!jupyter nbconvert --to script "D:/Github/TweeterPy/get_tweet_stuff.ipynb" --no-prompt

[NbConvertApp] Converting notebook D:/Github/TweeterPy/get_tweet_stuff.ipynb to script
[NbConvertApp] Writing 8460 bytes to D:\Github\TweeterPy\get_tweet_stuff.py


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import Markdown

In [1]:
import json 
from datetime import datetime
import pandas as pd
from tweeterpy import TweeterPy
#twitter password  =AJ#~Z+cBUR&7r/ 

In [5]:
# from gspread_dataframe import set_with_dataframe

In [6]:
# import gspread

# credentials = {
#     "installed": {
#         "client_id": "117044127800-dg6q9197854uc0g375ul2d7569d8v9mj.apps.googleusercontent.com",
#         "project_id": "docs-382418",
#         "auth_uri": "https://accounts.google.com/o/oauth2/auth",
#         "token_uri": "https://oauth2.googleapis.com/token",
#         "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
#         "client_secret": "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",
#         "redirect_uris": ["http://localhost"],
#     }
# }

In [7]:
# gc, authorized_user = gspread.oauth_from_dict(credentials)

In [ ]:
twitter = TweeterPy()
twitter.get_liked_tweets('rngland')
# twitter.get_friends('rngland', following=True)

In [86]:
with open('liked_tweets.py', 'r') as file:
    file_contents = file.read()
my_dict = {}
exec(file_contents, my_dict)

In [84]:
rest_ids = []
created_at = []
profile_images = []
screen_names = []
name = []
full_text = []
expanded_urls = []
binding_values = []


for i in range(len(my_dict['liked']['data'])):
    
    # rest_id
    if 'rest_id' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        rest_ids.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id'])
    
    # legacy
    # created_at
    if 'legacy' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        created_at.append(datetime.strptime(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['created_at'], "%a %b %d %H:%M:%S %z %Y").strftime("%Y-%m-%d %H:%M:%S")) 
        full_text.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['full_text']) 
    # expanded_urls 
        if len(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']) >= 1:
            _1 = my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']        
            _2 = [p['expanded_url'] for p in _1]
            expanded_urls.append('  '.join(_2))
        else:
            expanded_urls.append(' ')
        
    # core
    # profile_images    
    if 'core' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        profile_images.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['profile_image_url_https']) 
    # screen_names
        screen_names.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['screen_name'])
    # name   
        name.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['name'])
    # binding_values
        if 'card' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
            binding_values.append((my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'],my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id']))  

    

p_img = ["=IMAGE(\"" + profile_images[i] + "\"" + ',4 ,45, 45)'  for i in range(len(profile_images))]

p_img = [image.replace("'", "") for image in p_img]

profile_url =['https://twitter.com/' + i for i in screen_names]

tweet_url =['https://twitter.com/' + screen_names[i] + '/status/' + rest_ids[i] for i in range(len(
screen_names))]


field_names = [
"rest_ids",
"created_at",
"profile_url",
"p_img",
"screen_names",
"name",
"full_text",
"expanded_urls",
"tweet_url",
# "card_title",
# "card_description",
# "card_thumbnail_image",
# "card_photo_image_full_size_alt_text"
]


data_dict = {field: eval(field) for field in field_names}
df = pd.DataFrame(data_dict)


card_title = [(p['value']['string_value'],i[1]) for i in binding_values for p in i[0] if p['key'] =='title']

card_description = [(p['value']['string_value'],i[1]) for i in binding_values for p in i[0] if p['key'] =='description']

card_thumbnail_image = [(p['value']['image_value']['url'],i[1]) for i in binding_values for p in i[0] if p['key'] =='thumbnail_image']

card_photo_image_full_size_alt_text = [(p['value']['string_value'],i[1]) for i in binding_values for p in i[0] if p['key'] =='photo_image_full_size_alt_text']

card_list = ["card_title", "card_description", "card_thumbnail_image", "card_photo_image_full_size_alt_text"]
card_dict = {card_type: eval(card_type) for card_type in card_list}


df.rest_ids = df.rest_ids.astype('int64')
dataframes = [pd.DataFrame(v, columns=[f"{k}", 'id']) for k, v in card_dict.items()]
dataframes = [df.astype({'id': 'int64'}) for df in dataframes]
dataframes = [df.set_index("id") for df in dataframes]
final_df = reduce(lambda x, y: pd.merge(x, y, left_on='rest_ids', how='left', right_index=True), dataframes, df)


[f"{i} :  {len(eval(i))}" for i in field_names]

final_df



['rest_ids :  664',
 'created_at :  664',
 'profile_url :  664',
 'p_img :  664',
 'screen_names :  664',
 'name :  664',
 'full_text :  664',
 'expanded_urls :  664',
 'tweet_url :  664']

,rest_ids,created_at,profile_url,p_img,screen_names,name,full_text,expanded_urls,tweet_url,card_title,card_description,card_thumbnail_image,card_photo_image_full_size_alt_text
0,1705155679819084179,2023-09-22 09:42:48,https://twitter.com/tdinh_me,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",tdinh_me,Tony Dinh 🎯,My full story: \n\nhttps://t.co/SXBdF7Th2B,https://news.tonydinh.com/p/my-solopreneur-sto...,https://twitter.com/tdinh_me/status/1705155679...,My solopreneur story: zero to $45K/mo in 2 years,Today is exactly 2 years since I quit my job a...,https://pbs.twimg.com/card_img/170515561890523...,NaN
1,1700004016506216614,2023-09-08 04:31:56,https://twitter.com/theJordanNoone,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",theJordanNoone,Jordan Noone 🇺🇸,@ryxcommar meanwhile the best coders write the...,https://python-pptx.readthedocs.io/en/latest/,https://twitter.com/theJordanNoone/status/1700...,NaN,NaN,NaN,NaN
2,1704989552459579398,2023-09-21 22:42:41,https://twitter.com/mayfer,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",mayfer,murat 🍥,scammers salivating harder by the hour,,https://twitter.com/mayfer/status/170498955245...,NaN,NaN,NaN,NaN
3,1704698971996803138,2023-09-21 03:28:01,https://twitter.com/cto_junior,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",cto_junior,TDM (e/flλ),@up_61347132 @sog_on_bird_app @deepfates https...,https://github.com/deepfates/memery,https://twitter.com/cto_junior/status/17046989...,GitHub - deepfates/memery: Search over large i...,Search over large image datasets with natural ...,https://pbs.twimg.com/card_img/170451302939874...,Search over large image datasets with natural ...
4,1704191384435990713,2023-09-19 17:51:03,https://twitter.com/gdb,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",gdb,Greg Brockman,"Get paid to red team our models, using your do...",https://openai.com/blog/red-teaming-network,https://twitter.com/gdb/status/170419138443599...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,1587867700231081987,2022-11-02 18:02:15,https://twitter.com/NoahTopper,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",NoahTopper,Noah Topper 🔍 (in Seattle 5/6 - 9/29),I’m reading “What Computers Still Can’t Do” an...,,https://twitter.com/NoahTopper/status/15878677...,NaN,NaN,NaN,NaN
660,1584659361632161792,2022-10-24 21:33:28,https://twitter.com/radshaan,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",radshaan,Ishan,Every single talented engineer (electrical/sof...,,https://twitter.com/radshaan/status/1584659361...,NaN,NaN,NaN,NaN
661,1552124372231458817,2022-07-27 02:51:02,https://twitter.com/BenHanowell,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",BenHanowell,Ben Hanowell 🦜☕,@PhDemetri @baursafi It’s most popular data ma...,,https://twitter.com/BenHanowell/status/1552124...,NaN,NaN,NaN,NaN
662,1499838567111143431,2022-03-04 20:05:54,https://twitter.com/tyba_latam,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",tyba_latam,tyba,"¡Invierte tu dinero, no juegues con él! tyba, ...",,https://twitter.com/tyba_latam/status/14998385...,NaN,NaN,NaN,NaN


In [49]:
# sh = gc.open("liked_tweets")

# worksheet  = sh.get_worksheet(0)    

# worksheet.format(f"g2:g{len(rest_ids)}", {
#   "textFormat": {
#     "fontSize": 14
#    }
# })
# set_with_dataframe(worksheet, df) 

In [ ]:
print(my_dict['liked']['data'][8]['content']['itemContent']['tweet_results']['result']['legacy']['full_text'])

urls = my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']

for i in urls:
    i['expanded_url']

# [i for i in urls['expanded_url']]

screen_name = my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['screen_name']

rest_id = my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['rest_id']

'https://twitter.com/' + screen_name + '/status/' + rest_id

screen_name
rest_id

print(my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['description'])


my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['legacy']['created_at']

#  'in_reply_to_screen_name': 'ryxcommar',
# 'in_reply_to_status_id_str': '1700002950012145747',

# my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['is_blue_verified']

my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['profile_image_url_https']


my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'][2]['value']['string_value']



# my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values']

# my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'][4]
# my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'][0]['value']['string_value']


# card  
    #   {'key': 'title',
    #        'value': {'string_value': 'My solopreneur story: zero to $45K/mo in 2 years',
    #   {'key': 'description',
    #        'value': {'string_value': 'Today is exactly 2 years since I quit my job and become a full-time indie hacker.',
        #   {'key': 'photo_image_full_size_alt_text',
        #    'value': {'string_value': 'Search over large image datasets with natural language and computer vision! - GitHub - deepfates/memery: Search over large image datasets with natural language and computer vision!',
        #      {'key': 'thumbnail_image',
        #    'value': {'image_value': {'height': 200,
        #      'width': 400,
        #      'url': 'https://pbs.twimg.com/card_img/1704513029398745088/aRyYEIOj?format=jpg&name=400x400'},
        #     'type': 'IMAGE'}},
        ##card 

I want to share my non-rigorous vibes based approach towards data science modeling of data. A lot of my job as a staff data scientist (i'm staff btw) is doing a vibe-check on technical problems, and helping other data scientists from going down an intractable path.
Indie hacker.

🧠 @TypingMindApp $30K/mo
📸 @XnapperHQ $6K/mo
🛠 @devutils_app $5.5K/mo
🪄 @blackmagic_so (Sold, was $14K/mo)

💬 https://t.co/9b1SSybSdG


'Today is exactly 2 years since I quit my job and become a full-time indie hacker.'